In [0]:
import tensorflow as tf
from tensorflow.keras import models, layers
# As per Co-training, all leaky Relu has slope of 0.01
lrelu = lambda features : tf.nn.leaky_relu(features, alpha=0.01)

def MobileNet():
    model = models.Sequential()
    model.add(layers.Conv2D(32, 3, strides=2, padding='same', input_shape=(224, 224, 3)))
    model.add(layers.DepthwiseConv2D(32, 1,padding='same'))
    model.add(layers.Conv2D(64, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(64, 2, padding='same'))
    model.add(layers.Conv2D(128, 1, strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(128, 1,padding='same'))

    model.add(layers.Conv2D(128, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(128, 2,padding='same'))
    model.add(layers.Conv2D(256, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(256, 1,padding='same'))
    model.add(layers.Conv2D(256, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(256, 2,padding='same'))

    model.add(layers.Conv2D(512, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(512, 1,padding='same'))
    model.add(layers.Conv2D(512, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(512, 1,padding='same'))
    model.add(layers.Conv2D(512, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(512, 1,padding='same'))
    model.add(layers.Conv2D(512, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(512, 1,padding='same'))
    model.add(layers.Conv2D(512, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(512, 1,padding='same'))
    model.add(layers.Conv2D(512, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(512, 2,padding='same'))

    model.add(layers.Conv2D(1024, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(1024, 1,padding='same'))
    model.add(layers.Conv2D(1024, (1, 1), strides=1, padding='same'))
    model.add(layers.AveragePooling2D(pool_size=(7,7), strides=1, padding="valid"))
    model.add(layers.Dense(1024, activation=tf.nn.relu))
    model.add(layers.Dense(1000, activation=tf.nn.softmax))
    return model

In [47]:
MobileNet().summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_146 (Conv2D)          (None, 112, 112, 32)      896       
_________________________________________________________________
depthwise_conv2d_127 (Depthw (None, 112, 112, 32)      32800     
_________________________________________________________________
conv2d_147 (Conv2D)          (None, 112, 112, 64)      2112      
_________________________________________________________________
depthwise_conv2d_128 (Depthw (None, 56, 56, 64)        262208    
_________________________________________________________________
conv2d_148 (Conv2D)          (None, 56, 56, 128)       8320      
_________________________________________________________________
depthwise_conv2d_129 (Depthw (None, 56, 56, 128)       2097280   
_________________________________________________________________
conv2d_149 (Conv2D)          (None, 56, 56, 128)     

https://blog.csdn.net/u011974639/article/details/79199306

https://arxiv.org/pdf/1704.04861.pdf

https://keras-zh.readthedocs.io/layers/convolutional/